# Extracting time series for each ROI
--------------------------------------
- This will take the already clean nifti files (created using sciprt in the /cleaning folder), and extract timecourse per ROI
- Currently using Yeo parcellation, but can be changed easily


In [8]:
import pandas as pd 
from nilearn.maskers import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
from nilearn import datasets
import numpy as np
import nilearn.plotting
import os
import glob
import nilearn
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
%config Completer.use_jedi = False

In [27]:
# set output dir
output_dir = '/gpfs/gibbs/pi/levy_ifat/Or/ZivResults/timecourse/'

In [4]:
# load atlas
yeo = datasets.fetch_atlas_yeo_2011()


Dataset created in /home/oad4/nilearn_data/yeo_2011



 ...done. (0 seconds, 0 min)
Extracting data from /home/oad4/nilearn_data/yeo_2011/d7a5390bfb7686fb41fa64cc2ba058d0/Yeo_JNeurophysiol11_MNI152.zip..... done.


In [9]:
# create masker to extract functional data within atlas parcels
masker = NiftiLabelsMasker(labels_img=yeo['thick_17'], standardize=True,
                           memory='nilearn_cache')

In [10]:
# grab functional files (cleaned one)
func_files = glob.glob('/gpfs/gibbs/pi/levy_ifat/Or/ZivResults/afni_cleaning/_subject_id_*/cleanAfni/sub-*_ses-1_task-rest_space-MNI152NLin2009cAsym_desc-smoothAROMAnonaggr_bold_tproject.nii.gz')
len(func_files)

In [13]:
a = masker.fit_transform(func_files[0])

In [24]:
pd.DataFrame(a)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.016589,0.402362,-0.145686,0.771823,-0.736089,-0.346760,0.559376,0.875638,-0.477773,-0.325121,0.804702,0.616430,0.972926,-0.061920,-0.223909,0.322921,-0.190721
1,-0.158328,-0.292196,-0.481824,-0.051306,-0.433125,-0.982007,-0.673089,-0.025100,-0.520072,0.269416,1.253946,0.711284,1.133452,0.649335,-0.644452,0.521763,0.150516
2,-0.491469,-0.413150,-0.122037,-1.010513,0.599282,-0.902338,-1.524095,-0.790853,0.220953,1.107359,1.281905,0.934889,0.910733,1.687555,-0.403813,0.626669,0.870462
3,-0.070303,0.462662,0.528886,-0.974913,1.629922,0.025667,-1.079365,-0.598106,0.995475,1.417271,1.046679,0.841993,0.307091,2.363219,0.202776,0.250854,1.028311
4,1.106545,1.515064,0.695491,0.244743,1.805522,1.023365,0.335264,0.277705,0.887679,0.757232,0.737130,0.167509,-0.502176,1.845109,0.561419,-0.449262,0.084872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.684432,-0.209676,-0.129780,1.162672,-0.876934,0.585983,0.604388,-0.007453,-0.413317,-1.125263,-0.608119,-1.004993,-0.774720,-0.027001,-0.881342,-1.031511,-0.728681
296,-0.410541,-0.861314,0.034828,0.883661,-0.592307,0.133165,0.185173,-0.948725,-0.027232,-1.165414,-0.798902,-0.852944,-0.993814,-1.051227,-0.534310,-0.419836,-0.624458
297,-1.345732,-0.805368,0.450327,0.355528,-0.180899,-0.005910,0.161748,-0.970225,0.514404,-0.795052,-0.884154,-0.189464,-0.753214,-1.285827,0.146670,0.210339,-0.033077
298,-1.321925,-0.017230,0.743093,0.341481,-0.057146,0.210653,0.681454,-0.038359,0.670265,-0.479455,-0.601290,0.444348,-0.159791,-0.919468,0.626539,0.452993,0.297954


In [30]:
def masking(funcFile):
    sub = os.path.basename(funcFile).split('_')[0]
    # takes functional file and returns Pandas dataframe for average timeseries of each of these ROIs
    a = masker.fit_transform(funcFile[0])
    subDF = pd.DataFrame(a)
    # save it
    subDF.to_csv(f'{output_dir}{sub}_yeo-17.csv', index=False)
    return subDF

In [ ]:
# run parallel
results = Parallel(n_jobs=8)(delayed(masking)(funcFile) for funcFile in func_files)

In [5]:
# set connectivity to correlation (can be set to partial correlation)
connectome_measure = ConnectivityMeasure(kind='correlation')

{'description': "Yeo 2011 Atlas\n\n\nNotes\n-----\nThis atlas provides a labeling of some cortical voxels in the MNI152\nspace.\n\nFour versions of the atlas are available, according to the cortical\nmodel (thick or thin cortical surface) and to the number of regions\nconsidered (7 or 17).\n\nContent\n-------\n    :'anat': Background anatomical image for reference and visualization\n    :'thin_7': Cortical parcelation into 7 regions, thin cortical model\n    :'thin_17': Cortical parcelation into 17 regions, thin cortical model\n    :'thick_7': Cortical parcelation into 17 regions, thick cortical model\n    :'thick_17': Cortical parcelation into 17 regions, thick cortical model\n    :'colors_7': Text file for the coloring of 7-regions parcellation\n    :'colors_17': Text file for the coloring of 17-regions parcellation\n\n\nReferences\n----------\nFor more information on this dataset's structure, see\nhttp://surfer.nmr.mgh.harvard.edu/fswiki/CorticalParcellation_Yeo2011\n\nYeo BT, Krien